##Seq2Seq
Seq2Seq에는 심볼이 필요합니다. 코드를 짜기 앞서 디코터 입력의시작, 디코더 입력의 끝, 의미없는 심볼을 각각 'S' , 'E' , 'P'로 정해두겠습니다.

In [1]:
import tensorflow as tf
import numpy as np

char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxy단어나무놀이소녀키스사랑']
num_dic = {n:i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

seq_data = [['word','단어'],
            ['wood','나무'],
            ['game','놀이'],
            ['girl','소녀'],
            ['kiss','키스'],
            ['love','사랑']]

앞서 했던 것과 같이 알파벳과 우리가 쓸 한글을 넣어 줍시다.

In [0]:
def make_batch(seq_data):
  input_batch = []
  output_batch = []
  target_batch = []

  for seq in seq_data:
    input = [num_dic[n] for n in seq[0]]
    output = [num_dic[n] for n in ('S' + seq[1])]
    target = [num_dic[n] for n in (seq[1]+ 'E')]

    input_batch.append(np.eye(dic_len)[input])
    output_batch.append(np.eye(dic_len)[output])
    target_batch.append(target)

  return input_batch, output_batch, target_batch

```python
input = [num_dic[n] for n in seq[0]]
```
인코더 셀의 입력값을 위해 입력 단어를 한 글자씩 데어 배열로 만듭니다.

```python
 output = [num_dic[n] for n in ('S' + seq[1])]
 ```
 디코더 셀의 입력값을 위해 출력 단어의 글자들릉ㄹ 배열로 만들고, 시작을 나타내는 심볼 'S'를 맨 앞에 붙입니다.

 ```python
  target = [num_dic[n] for n in (seq[1]+ 'E')]
```
학습을 위해 비교할 디코더 셀의 출력값을 만들고, 출력의 끝을 알려주는 심볼 'E'를 마지막에 붙입니다.


In [0]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 100

n_class = n_input = dic_len

다음으로 신경망 모델에서 사용할 하이퍼 파라미터, placeholder, 입출력 변수용 수치들을 정의 합니다.

In [0]:
enc_input = tf.placeholder(tf.float32, [None,None, n_input])
dec_input = tf.placeholder(tf.float32, [None,None, n_input])
targets = tf.placeholder(tf.int64, [None,None])

RNN의 특성상 입력데이터의 단계가 있습니다. 또한 입력값들은 원-핫 인코딩을 사용하고 디코더의 출력값은 인덱스 숫자를 그대로 사용하기 때문에 입력값의 차원이 하나더 높습니다.    
입력 형식: [batch size, time steps, input size]   
출력 형식: [batch size, time steps]

In [5]:
with tf.variable_scope('encode'):
  enc_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
  enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)

  outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype= tf.float32)

with tf.variable_scope('decode'):
  dec_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
  dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob = 0.5)

  outputs , dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state=enc_states, dtype= tf.float32)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Seq2Seq은 인코더와 디코더로 구분되므로 cell역시 인코더 셀과 디코더 셀을 구분하여 만들어야 합니다.   
셀은 기본셀(BasicLSTMCell)을 사용 하였고 각 셀에 드롭아웃도 적용했습니다.   
주의할 점은 디코더의 초기 상태 값(**입력값 아님**)으로 인코더의 최종 상태 값을 넣어줘야 한다는 것입니다.(**initial_state = enc_states**)


In [6]:
model = tf.layers.dense(outputs, n_class, activation = None)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits = model, labels = targets))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


출력층을 만들고 손실함수와 최적화 함수를 구성합니다.   
출력층은 layer모듈의 dense 함수를 사용하였습니다.   
Dense 를 사용할시: output = activation(dot(input, kernel) + bias)   
고수준의 API를 사용하면 가중치와 편향을 설정하지 않고도 알아서 처리해줍니다.

In [12]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, targets_batch = make_batch(seq_data)

for epoch in range(total_epoch):
  _,loss = sess.run([optimizer, cost], feed_dict= {enc_input: input_batch, dec_input: output_batch, targets: targets_batch})

  print('Epoch:', '%04d' %(epoch+1), 'cost=', '{:.6f}'.format(loss))

print('최적화 완료')

Epoch: 0001 cost= 3.687104
Epoch: 0002 cost= 3.570867
Epoch: 0003 cost= 3.356566
Epoch: 0004 cost= 2.936076
Epoch: 0005 cost= 2.368010
Epoch: 0006 cost= 2.318052
Epoch: 0007 cost= 2.006325
Epoch: 0008 cost= 1.834133
Epoch: 0009 cost= 1.913274
Epoch: 0010 cost= 1.598610
Epoch: 0011 cost= 1.458591
Epoch: 0012 cost= 1.159043
Epoch: 0013 cost= 1.177838
Epoch: 0014 cost= 0.867390
Epoch: 0015 cost= 0.856701
Epoch: 0016 cost= 0.778134
Epoch: 0017 cost= 0.703400
Epoch: 0018 cost= 0.583874
Epoch: 0019 cost= 0.409060
Epoch: 0020 cost= 0.540391
Epoch: 0021 cost= 0.343857
Epoch: 0022 cost= 0.348853
Epoch: 0023 cost= 0.265001
Epoch: 0024 cost= 0.411350
Epoch: 0025 cost= 0.267568
Epoch: 0026 cost= 0.349986
Epoch: 0027 cost= 0.183616
Epoch: 0028 cost= 0.325890
Epoch: 0029 cost= 0.154015
Epoch: 0030 cost= 0.198595
Epoch: 0031 cost= 0.139012
Epoch: 0032 cost= 0.121018
Epoch: 0033 cost= 0.202327
Epoch: 0034 cost= 0.194385
Epoch: 0035 cost= 0.086881
Epoch: 0036 cost= 0.119560
Epoch: 0037 cost= 0.092934
E

In [0]:
def translate(word):
  seq_data = [word,'P'*len(word)]

  input_batch, output_batch, targets_batch = make_batch([seq_data])

  prediction = tf.argmax(model,2)

  result = sess.run(prediction, feed_dict= {enc_input: input_batch, dec_input: output_batch, targets: targets_batch})

  decoded = [char_arr[i] for i in result[0]]

  end = decoded.index('E')
  translated = ''.join(decoded[:end])
  return translated

이 모델은 입력값과 출력값 데이터로 [영어단어, 한글뜻]를 사용하지만 예측시에는 한글뜻을 모르므로 디코더의 입출력을 의미없는 값인 'P'로 채워 데이터를 구성합니다.   
input_batch는 [ 'w', 'o', 'r', 'd']   
output_batch는 [ 'P, 'P', 'P', 'P'] 글자들의 인덱스를 원-핫 인코딩한 값   
target_batch는 [ 'P, 'P', 'P', 'P'] 의 각 글자의 인덱스인 [2, 2, 2, 2]가 됩니다.   

그리고 예측모델을 돌립니다. 세번재 차원을 argmax로 취해 가장 확률이 높은 글자(의 인덱스)를 예측값으로 만듭니다. 세번재 차원을 argmax로 취하는 이유는 결과값이 [batch_size, time steps, input size]형태로 나오기 때문입니다.


In [16]:
print('\n===번역테스트===')
print('word ->', translate('word'))
print('wodr ->', translate('wodr'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('abcd ->', translate('abcd'))
print('gisl ->', translate('gisl'))


===번역테스트===
word -> 단어
wodr -> 나어
love -> 사랑
loev -> 사랑
abcd -> 놀이
gisl -> 소녀


학습시킨 단어와 약간의 오타에도 그럴듯 하게 번역이 되었습니다.   
또한 이상한 단어에도 연관 없어보이지만 그럴듯한 결과를 추측하였습니다.   
이 모델에서 글자들을 단어로 바꾸기만 하면 문장 단위의 번역을, 그 후 입력을 질문으로, 출력을 답변으로 하는 데이터를 학습시키면 챗봇을 만들 수 있습니다.